# Accessing FVCOM Hindcast on AWS Public Data

Rich Signell Shared this repository and notebook with a quick demonstration of how to access FVCOM hindcast data on AWS public data.
https://github.com/OpenScienceComputing/umassd-fvcom/blob/main/fvcom_gom3_explore.ipynb

I am revisiting it here in this notebook for a test-case in extracting daily vertically integrated currents for two transects along the Maine Coastline.

The registry for the data can be found here:
https://registry.opendata.aws/fvcom_gom3/

In [ ]:
import xarray as xr
import zarr
import intake
import intake_xarray
import fastparquet

In [2]:
# ?
intake_catalog_url = 'https://umassd-fvcom.s3.amazonaws.com/gom3/hindcast/umassd-fvcom-gom3.yml'
cat = intake.open_catalog(intake_catalog_url)
list(cat)

['UMASSD-FVCOM-GOM3-zarr2', 'UMASSD-FVCOM-GOM3-zarr3']

In [3]:
if zarr.__version__[0]=='2':
    dataset = 'UMASSD-FVCOM-GOM3-zarr2' 
elif zarr.__version__[0]=='3':
    dataset = 'UMASSD-FVCOM-GOM3-zarr3'

In [ ]:
# Open/access the hindcast dataset
%%time
ds = cat[dataset].to_dask()
ds

CPU times: user 1.45 s, sys: 71 ms, total: 1.52 s
Wall time: 2.37 s


<xarray.Dataset> Size: 45TB
Dimensions:             (time: 341881, node: 48451, four: 4, nele: 90415,
                         three: 3, siglev: 46, maxnode: 11, maxelem: 9,
                         siglay: 45)
Coordinates:
    siglay              (siglay, node) float32 9MB dask.array<chunksize=(45, 48451), meta=np.ndarray>
    siglev              (siglev, node) float32 9MB dask.array<chunksize=(46, 48451), meta=np.ndarray>
  * time                (time) datetime64[ns] 3MB 1978-01-01 ... 2017-01-01
    x                   (node) float32 194kB dask.array<chunksize=(48451,), meta=np.ndarray>
    y                   (node) float32 194kB dask.array<chunksize=(48451,), meta=np.ndarray>
Dimensions without coordinates: node, four, nele, three, maxnode, maxelem
Data variables: (12/49)
    ML_Depth            (time, node) float32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
    ML_Nsiglay          (time, node) int32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
    a1u                 (four, nele) float32 1MB dask.array<chunksize=(4, 90415), meta=np.ndarray>
    a2u                 (four, nele) float32 1MB dask.array<chunksize=(4, 90415), meta=np.ndarray>
    art1                (node) float32 194kB dask.array<chunksize=(48451,), meta=np.ndarray>
    art2                (node) float32 194kB dask.array<chunksize=(48451,), meta=np.ndarray>
    ...                  ...
    wet_nodes           (time, node) int32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
    wet_nodes_prev_int  (time, node) int32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
    ww                  (time, siglay, nele) float32 6TB dask.array<chunksize=(1, 45, 90415), meta=np.ndarray>
    xc                  (nele) float32 362kB dask.array<chunksize=(90415,), meta=np.ndarray>
    yc                  (nele) float32 362kB dask.array<chunksize=(90415,), meta=np.ndarray>
    zeta                (time, node) float32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
Attributes: (12/14)
    title:                       GOM3 Nesting  by Yf.Sun@umassd.edu
    institution:                 School for Marine Science and Technology
    source:                      FVCOM_3.0
    history:                     model started at: 22/12/2011   16:41
    references:                  http://fvcom.smast.umassd.edu, http://codfis...
    Conventions:                 CF-1.0
    ...                          ...
    Tidal_Forcing:               TIDAL ELEVATION FORCING IS OFF!
    River_Forcing:               THERE ARE 50 RIVERS IN THIS MODEL.\nRIVER IN...
    GroundWater_Forcing:         GROUND WATER FORCING IS OFF!
    Surface_Heat_Forcing:        FVCOM variable surface heat forcing file:\nF...
    Surface_Wind_Forcing:        FVCOM variable surface Wind forcing:\nFILE N...
    Surface_PrecipEvap_Forcing:  FVCOM periodic surface precip forcing:\nFILE...

In [ ]:
# This is the range of dates available, very cool its just one huge thing
ds.time.tail

<bound method DataArray.tail of <xarray.DataArray 'time' (time: 341881)> Size: 3MB
array(['1978-01-01T00:00:00.000000000', '1978-01-01T01:01:52.499999744',
       '1978-01-01T01:58:07.500000256', ..., '2016-12-31T22:01:52.499999744',
       '2016-12-31T22:58:07.500000256', '2017-01-01T00:00:00.000000000'],
      shape=(341881,), dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 3MB 1978-01-01 ... 2017-01-01
Attributes:
    format:     modified julian day (MJD)
    long_name:  time
    time_zone:  UTC>